In [1]:
from DataLoader import MyOwnDataloader
from pycocotools.coco import COCO

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import snntorch as snn
from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt

from functools import partial
from dataclasses import dataclass
from collections import OrderedDict

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved

from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total/1000000}')
print(f'free     : {info.free/1000000}')
print(f'used     : {info.used/1000000}')

total    : 12636.061696
free     : 11462.90176
used     : 1173.159936


In [4]:

dataDir='/media/gamedisk/COCO_dataset/'
val='val2017'
train = 'train2017'

val_annFile='{}/annotations/instances_{}.json'.format(dataDir,val)
train_annFile='{}/annotations/instances_{}.json'.format(dataDir,train) 
# Batch size
batch_size = 1



classes = {
    "bird": 1,
    "cat": 2,
    "dog": 3,
    "horse": 4,
    "sheep": 5,
    "cow": 6,
    "elephant": 7,
    "bear": 8,
    "zebra": 9,
    "giraffe": 10
}
coco = COCO(val_annFile)
val_loader = MyOwnDataloader(dataDir = dataDir, dataType = val,
                     annFile = val_annFile, classes = classes, train_batch_size=batch_size)
valid_dl = val_loader.concat_datasets()


coco = COCO(train_annFile)
train_loader = MyOwnDataloader(dataDir = dataDir, dataType = train,
                     annFile = train_annFile, classes = classes, train_batch_size=batch_size)
train_dl = val_loader.concat_datasets()



loading annotations into memory...
Done (t=0.37s)
creating index...
index created!
loading annotations into memory...
Done (t=0.33s)
creating index...
index created!
loading annotations into memory...
Done (t=8.82s)
creating index...
index created!
loading annotations into memory...
Done (t=9.40s)
creating index...
index created!


In [5]:
spike_grad = surrogate.fast_sigmoid(slope=25)
beta = 0.5
num_steps = 50

Padded convolutional layer. Required to make residual connections

In [6]:
class Conv2dAuto(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding =  (self.kernel_size[0] // 2, self.kernel_size[1] // 2) # dynamic add padding based on the kernel_size
        
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False)      

ResNet Architecture, with Leaky I&F as an activation function instead of ReLU.

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.in_channels, self.out_channels =  in_channels, out_channels
        self.blocks = nn.Identity()
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut: residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels

class ResNetResidualBlock(ResidualBlock):
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, conv=conv3x3, *args, **kwargs):
        super().__init__(in_channels, out_channels)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv
        self.shortcut = nn.Sequential(OrderedDict(
        {
            'conv' : nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1,
                      stride=self.downsampling, bias=False),
            'bn' : nn.BatchNorm2d(self.expanded_channels)
            
        })).to(device) if self.should_apply_shortcut else None
        
        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels



In [8]:

def conv_bn(in_channels, out_channels, conv, *args, **kwargs):
    return nn.Sequential(OrderedDict({'conv': conv(in_channels, out_channels, *args, **kwargs), 
                          'bn': nn.BatchNorm2d(out_channels) })).to(device)



In [9]:
class ResNetBasicBlock(ResNetResidualBlock):
    expansion = 1
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv_bn(self.in_channels, self.out_channels, conv=self.conv, bias=False, stride=self.downsampling),
            snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
            conv_bn(self.out_channels, self.expanded_channels, conv=self.conv, bias=False),
        ).to(device)


In [10]:
class ResNetBottleNeckBlock(ResNetResidualBlock):
    expansion = 4
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, expansion=4, *args, **kwargs)
        self.blocks = nn.Sequential(
           conv_bn(self.in_channels, self.out_channels, self.conv, kernel_size=1),
             snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
             conv_bn(self.out_channels, self.out_channels, self.conv, kernel_size=3, stride=self.downsampling),
             snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
             conv_bn(self.out_channels, self.expanded_channels, self.conv, kernel_size=1),
        ).to(device)
    

In [11]:
class ResNetLayer(nn.Module):
    def __init__(self, in_channels, out_channels, block=ResNetBasicBlock, n=1, *args, **kwargs):
        super().__init__()
        # 'We perform downsampling directly by convolutional layers that have a stride of 2.'
        downsampling = 2 if in_channels != out_channels else 1
        
        self.blocks = nn.Sequential(
            block(in_channels , out_channels, *args, **kwargs, downsampling=downsampling),
            *[block(out_channels * block.expansion, 
                    out_channels, downsampling=1, *args, **kwargs) for _ in range(n - 1)]
        ).to(device)

    def forward(self, x):
        x = self.blocks(x)
        return x



In [12]:
class ResNetEncoder(nn.Module):
    """
    ResNet encoder composed by increasing different layers with increasing features.
    """
    def __init__(self, in_channels=3, blocks_sizes=[64, 128, 256, 512], deepths=[2,2,2,2],  block=ResNetBasicBlock, *args,**kwargs):
        super().__init__()
        
        self.blocks_sizes = blocks_sizes
        
        self.gate = nn.Sequential(
            nn.Conv2d(in_channels, self.blocks_sizes[0], kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(self.blocks_sizes[0]),
            snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        ).to(device)
        
        self.in_out_block_sizes = list(zip(blocks_sizes, blocks_sizes[1:]))
        self.blocks = nn.ModuleList([ 
            ResNetLayer(blocks_sizes[0], blocks_sizes[0], n=deepths[0], 
                        block=block,  *args, **kwargs),
            *[ResNetLayer(in_channels * block.expansion, 
                          out_channels, n=n, 
                          block=block, *args, **kwargs) 
              for (in_channels, out_channels), n in zip(self.in_out_block_sizes, deepths[1:])]       
        ]).to(device)
        
        
    def forward(self, x):
        x = self.gate(x)
        for block in self.blocks:
            x = block(x)
        return x



In [13]:
class ResnetDecoder(nn.Module):
    """
    This class represents the tail of ResNet. It performs a global pooling and maps the output to the
    correct class by using a fully connected layer.
    """
    def __init__(self, in_features, n_classes):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool2d((1, 1)).to(device)
        self.decoder = nn.Linear(in_features, n_classes).to(device)

    def forward(self, x):
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.decoder(x)
        return x

In [14]:
class ResNet(nn.Module):
    
    def __init__(self, in_channels, n_classes, *args, **kwargs):
        super().__init__()
        self.encoder = ResNetEncoder(in_channels, *args, **kwargs).to(device)
        self.decoder = ResnetDecoder(self.encoder.blocks[-1].blocks[-1].expanded_channels, n_classes).to(device)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def loss(self, prob, bbox, label, gt_bbox, lmb=1.0):
        loss_cat = self.cat_loss(prob, label)
        lbl = label.view(-1, 1, 1).expand(label.size(0), 1, 4)
        mask = (label != 0).float().view(-1, 1, 1).expand(label.shape[0], 1, 4)
        loss_loc = self.loc_loss(gt_bbox * mask, bbox.gather(1, lbl).squeeze(1) * mask)
        loss = loss_cat + lmb * loss_loc
        return loss, loss_cat, loss_loc

In [25]:
model = ResNet(3, 10, block=ResNetBasicBlock, deepths=[2, 2, 2, 2]).to(device)
model

ResNet(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Leaky()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNetBasicBlock(
            (blocks): Sequential(
              (0): Sequential(
                (conv): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): Leaky()
              (2): Sequential(
                (conv): Conv2dAuto(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [19]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
def train(model, train_dataset, optimizer, args):
    model.train()
    num_batches = len(train_dataset) // args.batch_size
    indexes = np.random.shuffle(np.arange(len(train_dataset)))
    loss_all = 0
    loss_cat_all = 0
    loss_loc_all = 0
    accuracy = 0
    num_samples = 0
    for i in range(num_batches):
        imgs = []
        rects = []
        roi_idxs = []
        rela_locs = []
        cats = []
        for j in range(args.batch_size):
            img, rect, roi_idx_len, rela_loc, cat = train_dataset[i *
                                                                 args.batch_size+j]
            # print(img, rect, roi_idx_len, gt_rect, cat)
            imgs.append(img.unsqueeze(0))
            rects += rect
            rela_locs += rela_loc
            roi_idxs += ([j] * roi_idx_len)
            cats += cat
        imgs = torch.cat(imgs, dim=0)
        rects = np.array(rects)
        rela_locs = torch.FloatTensor(rela_locs)
        cats = torch.LongTensor(cats)
        # print(imgs, rects, roi_idxs, rela_locs, cats)
        if args.cuda:
            imgs = imgs.cuda()
            rela_locs = rela_locs.cuda()
            cats = cats.cuda()
        optimizer.zero_grad()
        prob, bbox = model.forward(imgs, rects, roi_idxs)
        loss, loss_cat, loss_loc = model.loss(prob, bbox, cats, rela_locs)
        loss.backward()
        optimizer.step()
        num_samples += len(cats)
        loss_all += loss.item() * len(cats)
        loss_cat_all += loss_cat.item() * len(cats)
        loss_loc_all += loss_loc.item() * len(cats)
        accuracy += (torch.argmax(prob.detach(), dim=-1) == cats).sum().item()
    return model, loss_all/num_samples, loss_cat_all/num_samples, loss_loc_all/num_samples, accuracy/num_samples


In [26]:
epochs = 2
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam
history = []
model.train()
history += fit_one_cycle(epochs, max_lr, model, train_dl, valid_dl, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

AttributeError: 'ResNet' object has no attribute 'training_step'

In [ ]:
# params = [p for p in net.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# len_dataloader = len(data_loader)
# for epoch in range(2):
#     net.train()
#     i = 0    
#     for  id_batch, (x_batch, y_batch) in enumerate(data_loader):
#         i += 1
#         # imgs = list(img.to(device) for img in imgs)
#         # annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
#         x_batch = x_batch[0].to(device)
#         x_batch = x_batch.unsqueeze(0)
#         loss_dict = net(x_batch)
#         # losses = sum(loss for loss in loss_dict.values())

#         optimizer.zero_grad()
#         losses.backward()
#         optimizer.step()

#         print(f'Iteration: {i}/{len_dataloader}, Loss: {losses}')

NameError: name 'losses' is not defined

In [ ]:
# def forward_pass(net, num_steps, data):
#   mem_rec = []
#   spk_rec = []
#   utils.reset(net)  # resets hidden states for all LIF neurons in net

#   for step in range(num_steps):
#       spk_out, mem_out = net(data)
#       spk_rec.append(spk_out)
#       mem_rec.append(mem_out)

#   return torch.stack(spk_rec), torch.stack(mem_rec)

In [ ]:
# def batch_accuracy(train_loader, net, num_steps):
#   with torch.no_grad():
#     total = 0
#     acc = 0
#     net.eval()

#     train_loader = iter(train_loader)
#     for data, targets in train_loader:
#       data = data.to(device)
#       targets = targets.to(device)
#       spk_rec, _ = forward_pass(net, num_steps, data)

#       acc += SF.accuracy_rate(spk_rec, targets) * spk_rec.size(1)
#       total += spk_rec.size(1)

#   return acc/total

In [ ]:
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, betas=(0.9, 0.999))
# num_epochs = 10
# test_acc_hist = []

# # training loop
# for epoch in range(num_epochs):

#     avg_loss = backprop.BPTT(model, dl, optimizer=optimizer, criterion=loss_fn,
#                             num_steps=num_steps, time_var=False, device=device)

#     print(f"Epoch {epoch}, Train Loss: {avg_loss.item():.2f}")

#     # Test set accuracy
#     test_acc = batch_accuracy(dl, model, num_steps)
#     test_acc_hist.append(test_acc)

#     print(f"Epoch {epoch}, Test Acc: {test_acc * 100:.2f}%\n")